In [31]:
%load_ext autoreload
%autoreload 2

In [69]:
import numpy as np
import pandas as pd
import torch

from pathlib import Path
from pydub import AudioSegment
from IPython.display import Audio
import torchaudio

In [33]:
vid_path = Path("dataset/train")
metadata = pd.read_csv("dataset/metadata.csv")
masks = Path("dataset/lip_masks")

In [34]:
video_paths = [video_path for video_path in vid_path.iterdir() if video_path.suffix == ".mp4"]

In [35]:
from train_wan_t2v import TensorDataset

Exception ignored in: <function Wave_write.__del__ at 0x7f78085bc700>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/wave.py", line 326, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/wave.py", line 444, in close
    self._ensure_header_written(0)
  File "/opt/conda/lib/python3.10/wave.py", line 467, in _ensure_header_written
    self._write_header(datasize)
  File "/opt/conda/lib/python3.10/wave.py", line 471, in _write_header
    self._file.write(b'RIFF')
ValueError: I/O operation on closed file.
/home/xabieririzar/DiffSynth-Studio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
ds = TensorDataset(base_path="dataset", metadata_path="dataset/metadata.csv", steps_per_epoch=1000, audio_path="dataset/audio", frames=60)

28762 videos in metadata.
28420 tensors cached in metadata.
28762 audio files in metadata.


In [48]:
ds[0]['audio_latents'].shape

torch.Size([1, 13931])

In [49]:
from transformers import Wav2Vec2Model
wav2vec_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
audio_1 = ds[0]['audio_latents']
audio_2 = ds[1]['audio_latents']

batch = torch.stack([audio_1, audio_2], dim=0)

In [84]:
ds[0]['prompt_emb']['context'].shape

torch.Size([1, 512, 4096])

In [79]:
batch = batch.squeeze(1)

In [81]:
batch.shape

torch.Size([2, 13931])

In [80]:
out = wav2vec_model(batch)

In [67]:
audio.shape

torch.Size([1, 13931])